## Compare the differences across different variables

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from scipy import stats
import warnings
warnings.simplefilter('ignore')

In [2]:
import statsmodels.api as sm

In [3]:
df = pd.read_csv(
    '../../../dataset-collection/auto-mpg/auto-mpg.csv', sep='\s+', header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower',
              'weight', 'acceleration', 'year', 'origin', 'name']
df.drop(df[df.horsepower == '?'].index, inplace=True)
df['horsepower'] = df.horsepower.astype(float)
df['brand'] = df.name.map(lambda x: x.split(' ')[0]
                          ).replace(
    {'toyouta': 'toyota',
     'maxda': 'mazda',
     'chevroelt': 'chevrolet',
     'vw': 'volkswagen',
     'vokswagen': 'volkswagen',
     'mercedes-benz': 'mercedes'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 10 columns):
mpg             392 non-null float64
cylinders       392 non-null int64
displacement    392 non-null float64
horsepower      392 non-null float64
weight          392 non-null float64
acceleration    392 non-null float64
year            392 non-null int64
origin          392 non-null int64
name            392 non-null object
brand           392 non-null object
dtypes: float64(5), int64(3), object(2)
memory usage: 33.7+ KB


In [4]:
cols = [col for col in df.columns if col not in ['origin','brand','name']]

In [5]:
grouped = df.groupby('origin')[cols].agg(np.mean)
grouped

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
origin,,,,,,,
1,20.033469,6.277551,247.512245,119.048980,3372.489796,14.990204,75.591837
2,27.602941,4.161765,109.632353,80.558824,2433.470588,16.794118,75.676471
3,30.450633,4.101266,102.708861,79.835443,2221.227848,16.172152,77.443038


In [6]:
statistic_12, pvalue_12 = (stats.ttest_ind(df[cols][df.origin==1], df[cols][df.origin==2]))
statistic_13, pvalue_13 = (stats.ttest_ind(df[cols][df.origin==1], df[cols][df.origin==3]))
statistic_23, pvalue_23 = (stats.ttest_ind(df[cols][df.origin==2], df[cols][df.origin==3]))

In [7]:
pd.DataFrame([statistic_12, pvalue_12, statistic_13, pvalue_13, statistic_23, pvalue_23],columns=cols, index=['statistic_12', 'pvalue_12', 'statistic_13', 'pvalue_13', 'statistic_23', 'pvalue_23']).T

,statistic_12,pvalue_12,statistic_13,pvalue_13,statistic_23,pvalue_23
mpg,-8.534456,6.306532e-16,-12.664889,4.172837e-30,-2.723325,0.007257
cylinders,10.390715,6.622661e-22,11.438380,1.181036e-25,0.660843,0.509762
displacement,11.460695,1.329399e-25,12.955018,3.506394e-31,1.824910,0.070073
horsepower,7.681404,2.065337e-13,8.460994,9.438940e-16,0.230937,0.817689
weight,9.250990,3.689302e-18,12.531001,1.300614e-29,3.139471,0.002051
acceleration,-4.674354,4.399322e-06,-3.556380,4.324324e-04,1.479140,0.141272
year,-0.171019,8.643201e-01,-3.911413,1.119470e-04,-3.010692,0.003076


Note that as long as we keep these tests independent, that works fine, but when we are looking for group differences across many variables until we find some, we might run into trouble. Therefore it is advisable to adjust for treating with multiple comparisons. In general, one will scale down the p-values of the individual tests which should lead to a "discovery". 

This can be done straight-forwardly with a variety of frequently used methods with statsmodels. We just pass a list of p-values. We obtain boolean values which are `True` for those hypotheses which can be rejected, the corrected p-values, and corrected significance level values.

In [8]:
sm.stats.multipletests(pvalue_12)

(array([ True,  True,  True,  True,  True,  True, False]),
 array([2.66453526e-15, 0.00000000e+00, 0.00000000e+00, 6.19504448e-13,
        0.00000000e+00, 8.79862494e-06, 8.64320079e-01]),
 0.007300831979014655,
 0.0071428571428571435)

The last Bonferroni corrected alpha is scaled down by the number of comparisons:

In [9]:
0.05/7

0.0071428571428571435

The Sidak corrected alpha is given by the following expression:

In [10]:
1-(1-0.05)**(1/7)

0.007300831979014655